In [2]:
import numpy as np

In [90]:
sign = np.vectorize(lambda x : 1 if x > 0 else -1)

In [136]:
class Hopfield :
    def __init__ (self,n_cells, memories):
        self.n_cells = n_cells
        rows, cols = memories.shape
        self.weights = (memories.T * memories - np.mat(np.eye(cols))*rows) / float(cols)
        self.memories = memories
        
    def finish(self):
        self.weights = self.weights * (1/float(self.n_cells))
        
    def energy(self, state):
        return np.asscalar(-1/float(2) * state * self.weights * state.T)
        
    def findMinimum(self,state):
        actual_state = state
        equals = False
        while not equals :
            next_state = sign(self.weights * actual_state.T).T 
            equals = np.all(np.equal(actual_state,next_state))
            actual_state = next_state
        return actual_state
    
    def isStable(self, error):
        stable_memories = [np.all(np.equal(memory,hn.findMinimum(memory))) for memory in hn.memories]
        errors = filter(lambda x: x == False, stable_memories)
        return (len(errors) / float(len(stable_memories))) < error

In [175]:
N = 700
stable = True
memories = []
while stable:
    memories.append(sign_function(np.random.uniform(-1,1,N)))
    hn = Hopfield(N,np.mat(memories))
    stable = hn.isStable(.1)
print len(memories) / float(N)

0.0857142857143
